In [1]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import h5py
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
%matplotlib
#%matplotlib qt

Using matplotlib backend: Qt5Agg


In [10]:
%cd '/scratch/tgreer/edge_coupler/data'
%ls

/scratch/tgreer/edge_coupler/data
coupler001.h5  coupler008.h5  coupler015.h5  coupler022.h5  coupler029.h5
coupler002.h5  coupler009.h5  coupler016.h5  coupler023.h5  coupler030.h5
coupler003.h5  coupler010.h5  coupler017.h5  coupler024.h5  coupler031.h5
coupler004.h5  coupler011.h5  coupler018.h5  coupler025.h5
coupler005.h5  coupler012.h5  coupler019.h5  coupler026.h5
coupler006.h5  coupler013.h5  coupler020.h5  coupler027.h5
coupler007.h5  coupler014.h5  coupler021.h5  coupler028.h5


In [11]:
class Sim:
    pass
sims = []
file_idx = 1
while(True):
    fname = f'coupler{file_idx:03d}.h5'
    try:
        with h5py.File(fname, 'r') as hf:
            sim = Sim()
            sim.attrs = { key:hf['simulation'].attrs[key] for key in hf['simulation'].attrs.keys() }
            sim.data = { key:hf['simulation'][key][:].squeeze() for key in hf['simulation'].keys() }
            sims.append(sim)

                
    except:
        break
    file_idx += 1

In [24]:
# find max E and H field magnitudes
maxE = 0; maxH = 0
for sim in sims:
    maxE = max(maxE, np.max(np.abs(sim.data['Ex'])))
    maxE = max(maxE, np.max(np.abs(sim.data['Ey'])))
    maxE = max(maxE, np.max(np.abs(sim.data['Ez'])))
    maxH = max(maxH, np.max(np.abs(sim.data['Hx'])))
    maxH = max(maxH, np.max(np.abs(sim.data['Hy'])))
    maxH = max(maxH, np.max(np.abs(sim.data['Hz'])))


In [26]:
@interact
def plot_func(file_idx=(0,len(sims)-1), which=sims[0].data.keys(), 
              component=['real', 'imag', 'mag'],
              normalize=True,
              mirror=True, grid=True):
        matplotlib.rcParams['figure.figsize'] = [20,5]
        data = sims[file_idx].data[which]
        if which=='eps':
            cmap = 'Blues'
            norm = None
        else:
            # field (e.g. Ex)
            cmap = 'seismic'
            if normalize:
                if which in ('Ex', 'Ey', 'Ez'):
                    dmax = maxE
                elif which in ('Hx', 'Hy', 'Hz'):
                    dmax = maxH
            else:
                dmax = np.max(np.abs(data))
            norm = matplotlib.colors.Normalize(-dmax, dmax)

        attrs = sims[file_idx].attrs
        dx,dy,dz = attrs['dx'], attrs['dy'], attrs['dz']
        X,Y,Z = attrs['X'], attrs['Y'], attrs['Z']
        if mirror:
            if which in ('Ex', 'Ez', 'Hy'):
                flip = -1
            else:
                flip = 1
            data = np.vstack((flip*data[-1:0:-1,:], data))
            extent = [-dx/2, X+dx/2, Y+dy/2, -Y-dy/2]
        else:
            extent = [-dx/2, X+dx/2, Y+dy/2, -dy/2]
                
        #plt.clf()
        if component=='real': data = data.real
        elif component=='imag': data=data.imag
        elif component=='mag': data=np.abs(data)
                
        plt.imshow(data, interpolation='none', cmap=cmap, extent=extent, norm=norm)
        if grid:
            plt.grid()
        plt.gca().set_aspect('equal')
        plt.colorbar()
        plt.title(which)
        plt.xlabel('x [microns]')
        plt.ylabel('y [microns]')


interactive(children=(IntSlider(value=15, description='file_idx', max=30), Dropdown(description='which', optio…